# Modeling
### [1] 원핫 없이 통으로 학습 & 종목명 원핫 적용 후 학습 비교
### [2] 머신러닝 => 전체 돌리는 모듈로 상위 3개 정도 & 딥러닝 모델 비교

In [1]:
import pandas as pd
import numpy as np
import os
import shutil
from tqdm import tqdm
from module import connection, get, save, search, update

In [19]:
stock_df = pd.DataFrame()

files_m = os.listdir('../data/test_label/')

for min_df in tqdm(files_m[:10]):
    _df = pd.read_csv(f'../data/test_label/{min_df}', index_col='Unnamed: 0')
    stock_df = pd.concat([stock_df, _df])

100%|██████████| 10/10 [00:05<00:00,  1.94it/s]


In [30]:
# 첫 5줄을 봅니다.
stock_df['저가'].value_counts()

170000    5811
169500    5663
126500    5340
122500    4935
126000    4922
          ... 
45681        1
16438        1
26092        1
16555        1
15392        1
Name: 저가, Length: 2636, dtype: int64

In [16]:
kospi = pd.read_csv('../data/kospi_20221110.csv', encoding='euc-kr')
kosdaq = pd.read_csv('../data/kosdaq_20221110.csv', encoding='euc-kr')

In [17]:
kospi = kospi[kospi['주식종류'] == '보통주']
kosdaq = kosdaq[kosdaq['주식종류'] == '보통주']

In [18]:
kospi = kospi.iloc[:,[1,-3]]
kosdaq = kosdaq.iloc[:,[1,-3]]

In [19]:
kospi.shape, kosdaq.shape

((824, 2), (1598, 2))

In [20]:
kospi_list = kospi.iloc[:,0]
kosdaq_list = kosdaq.iloc[:,0]

stock_list = pd.concat([kospi_list,kosdaq_list])

In [21]:
stock_list = stock_list.to_list()
stock_list

['095570',
 '006840',
 '282330',
 '027410',
 '138930',
 '001460',
 '001040',
 '011150',
 '000590',
 '012030',
 '016610',
 '005830',
 '000990',
 '139130',
 '001530',
 '001880',
 '000210',
 '375500',
 '155660',
 '069730',
 '017940',
 '365550',
 '383220',
 '007700',
 '006360',
 '078930',
 '012630',
 '294870',
 '097230',
 '014790',
 '204320',
 '060980',
 '003560',
 '175330',
 '234080',
 '001060',
 '096760',
 '105560',
 '432320',
 '009070',
 '016380',
 '001390',
 '033180',
 '001940',
 '025000',
 '092230',
 '000040',
 '093050',
 '034220',
 '003550',
 '051900',
 '373220',
 '032640',
 '011070',
 '066570',
 '051910',
 '079550',
 '010120',
 '000680',
 '006260',
 '108320',
 '001120',
 '023150',
 '035420',
 '181710',
 '338100',
 '034310',
 '008260',
 '004250',
 '010060',
 '010950',
 '005090',
 '001380',
 '001770',
 '002360',
 '009160',
 '123700',
 '025530',
 '011790',
 '018670',
 '001740',
 '210980',
 '395400',
 '034730',
 '402340',
 '361610',
 '096770',
 '001510',
 '285130',
 '017670',
 '064960',

In [29]:
files_list = os.listdir('../data/test/')

# f_list = []

for file in tqdm(files_list):    
    file_df = pd.read_csv(f'../data/test/{file}', index_col='Unnamed: 0')
    for stock in stock_list:
        if file.split('_')[-2] == stock:
            file_df.to_csv(f'../data/test1/{file}')


# for stock, f in zip(stock_list, f_list)

# ff_list = set(stock_list).intersection(set(f_list))
# ff_list = list(ff_list)
# for aa in ff_list:
#     print(search.search_by_code('A'+aa)[0])
#     shutil.move('../data/test/{0}_{1}.csv'.format(search.search_by_code(aa)[0][1:], search.search_by_code(aa)[1])
#     , '../data/test1/{0}_{1}.csv'.format(search.search_by_code(aa)[0][1:], search.search_by_code(aa)[1]))




100%|██████████| 100/100 [02:50<00:00,  1.71s/it]


In [40]:
import os
def update_kospi_kosdaq_list():
    kospi = pd.read_csv('../data/kospi_20221110.csv', encoding='euc-kr')
    kosdaq = pd.read_csv('../data/kosdaq_20221110.csv', encoding='euc-kr')

    kospi = kospi[kospi['주식종류'] == '보통주'].iloc[:,[1,-3]].iloc[:,0]
    kosdaq = kosdaq[kosdaq['주식종류'] == '보통주'].iloc[:,[1,-3]].iloc[:,0]

    
    kospi_kosdaq_list = pd.concat([kospi,kosdaq]).to_list()

    file_source = '../data/test/'
    file_destination = '../data/test2/'
    
    files_list = os.listdir(file_source)
    
    for file in tqdm(files_list):
        for k_k in kospi_kosdaq_list :
            if file.split('_')[-2] == k_k:
                os.replace(file_source + file, file_destination + file)

In [41]:
kospi_kosdaq_list()

100%|██████████| 100/100 [00:00<00:00, 612.91it/s]


In [24]:
stock_df.shape

(1897741, 25)

## 3. 원-핫 인코딩

In [25]:
# 속성을 X, 클래스를 y로 저장합니다.
X = stock_df.iloc[:,0:-1]
y = stock_df.iloc[:,-1]

# X와 y의 첫 5줄을 출력해 보겠습니다. 
print(X[0:5])
print(y[0:5])

            시간     시가     고가     저가     종가  전일대비   거래량       거래대금  누적체결매도수량  \
20221103  1530  26450  26450  26450  26450     0  9183  242880000     56069   
20221103  1520  26700  26700  26600  26700     0   603   16090000     56069   
20221103  1519  26600  26700  26550  26700     0   803   21380000     55878   
20221103  1518  26600  26600  26550  26600     0   166    4410000     55848   
20221103  1517  26550  26600  26550  26600     0  1048   27870000     55814   

          상장주식수  ...  외국인현보유비율  수정주가일자  수정주가비율  기관순매수량  기관누적순매수량  등락주선  등락비율  \
20221103      0  ...       0.0       0     0.0       0         0     0   0.0   
20221103      0  ...       0.0       0     0.0       0         0     0   0.0   
20221103      0  ...       0.0       0     0.0       0         0     0   0.0   
20221103      0  ...       0.0       0     0.0       0         0     0   0.0   
20221103      0  ...       0.0       0     0.0       0         0     0   0.0   

          예탁금  주식회전율  거래성립률  
20221103    0 

In [26]:
# 원-핫 인코딩 처리를 합니다.
y = pd.get_dummies(y)

# 원-핫 인코딩 결과를 확인합니다.
print(y[0:5])

          15469   15585   15624   15702   15740   15779   15818   15857   \
20221103       0       0       0       0       0       0       0       0   
20221103       0       0       0       0       0       0       0       0   
20221103       0       0       0       0       0       0       0       0   
20221103       0       0       0       0       0       0       0       0   
20221103       0       0       0       0       0       0       0       0   

          15896   15934   ...  470000  470500  471000  471500  472500  473000  \
20221103       0       0  ...       0       0       0       0       0       0   
20221103       0       0  ...       0       0       0       0       0       0   
20221103       0       0  ...       0       0       0       0       0       0   
20221103       0       0  ...       0       0       0       0       0       0   
20221103       0       0  ...       0       0       0       0       0       0   

          474000  474500  475500  477000  
20221103     

## 4. 소프트맥스

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# 모델 설정
model = Sequential()
model.add(Dense(12,  input_dim=4, activation='relu'))
model.add(Dense(8,  activation='relu')) # 주식에서는 relu를 쓰면 안된다.(변동률이 마이너스가 가능하기 때문에)
# 주식에서는 relu 대신에 linear를 쓰는 경우가 많다.
model.add(Dense(8,  activation='relu'))
model.add(Dense(8,  activation='relu'))
model.add(Dense(8,  activation='relu'))
model.add(Dense(3, activation='softmax')) # 답이랑 오차를 구하기 위해서 값의 총합을 1로 바꾸는 함수
model.summary()

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 실행
history=model.fit(X, y, epochs=30, batch_size=5)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 12)                60        
                                                                 
 dense_14 (Dense)            (None, 8)                 104       
                                                                 
 dense_15 (Dense)            (None, 8)                 72        
                                                                 
 dense_16 (Dense)            (None, 8)                 72        
                                                                 
 dense_17 (Dense)            (None, 8)                 72        
                                                                 
 dense_18 (Dense)            (None, 3)                 27        
                                                                 
Total params: 407
Trainable params: 407
Non-trainable 

## 5. 아이리스 품족 예측 실행

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 깃허브에 준비된 데이터를 가져옵니다. 앞에서 이미 가져왔으므로 주석 처리합니다. 5번 예제만 별도 실행 시 주석을 해제한 후 실행해주세요.
# !git clone https://github.com/taehojo/data.git

# 아이리스 데이터를 불러옵니다.
df = pd.read_csv('./data/iris3.csv')

# 속성을 X, 클래스를 y로 저장합니다.
X = df.iloc[:,0:4]
y = df.iloc[:,4]

# 원-핫 인코딩 처리를 합니다.
y = pd.get_dummies(y)

# 모델 설정
model = Sequential()
model.add(Dense(12,  input_dim=4, activation='relu'))
model.add(Dense(8,  activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 실행
history=model.fit(X, y, epochs=30, batch_size=5)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 12)                60        
_________________________________________________________________
dense_13 (Dense)             (None, 8)                 104       
_________________________________________________________________
dense_14 (Dense)             (None, 3)                 27        
Total params: 191
Trainable params: 191
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
30/30 [==============================] - 0s 2ms/step - loss: 1.4888 - accuracy: 0.3333
Epoch 2/30
30/30 [==============================] - 0s 2ms/step - loss: 1.0854 - accuracy: 0.4067
Epoch 3/30
30/30 [==============================] - 0s 2ms/step - loss: 0.9764 - accuracy: 0.6000
Epoch 4/30
30/30 [==============================] - 0s 2ms/step - loss: 0.9382 - accuracy: 0